In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
from sklearn.utils import shuffle
from sklearn.manifold import TSNE
import os
import pickle
import importlib
import detect_bursts
importlib.reload(detect_bursts)
import copy
import warnings
warnings.filterwarnings("ignore")
import seaborn as sns
from sklearn import preprocessing
from sklearn.decomposition import KernelPCA
from sklearn.cluster import KMeans
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.metrics import accuracy_score,roc_auc_score,f1_score,balanced_accuracy_score,confusion_matrix,ConfusionMatrixDisplay
from sklearn.model_selection import GridSearchCV,train_test_split,cross_val_predict,StratifiedGroupKFold,StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
#XGBOOST
from xgboost import XGBClassifier
import xgboost
from pygam import GAM, s, f, LogisticGAM
from collections import defaultdict
from sklearn import tree
import json
from scipy.io import loadmat
import optuna

Load Unlabeled Data

In [2]:
def convertIntoFloatList(str):
    #preprocess data extracted
    #remove the "[" and "]" characters
    str = str[1:-1]
    if str == '':
        return None
    return np.array(str.split(','),dtype = float)

def readData(frame,mothName):
    #convert dataframe rows into an entire list
    tp = []
    nameArr = []
    for ind in frame.index:
        inc = convertIntoFloatList(frame[ind])
        if inc is not None:
            tp.append(inc)
            nameArr.append(mothName)
    return tp,nameArr

#load xlsx data, extract column with largest pre_stim
def argMaxPreStim(mothNum):

    #load data from csv file
    loadPath =  f'~/Documents/GitHub/AL_generative/timestamps_{mothNum}.csv'

    df = pd.read_csv(loadPath,header=0)

    argMaxColName = df.columns[np.argmax(df.iloc[0][2:]) + 2]
    print(f"##########current stimuli referenced: {argMaxColName}##########")
    return argMaxColName

#load xlsx data, extract column with smallest pre_stim
def argMinPreStim(mothNum):

    #load data from csv file
    loadPath =  f'~/Documents/GitHub/AL_generative/ALdata/timestamps_{mothNum}.csv'

    df = pd.read_csv(loadPath,header=0)

    argMinColName = df.columns[np.argmin(df.iloc[0][2:]) + 2]
    print(f"##########current stimuli referenced: {argMinColName}##########")
    return argMinColName

#load csv data, extract timestamps
def loadData(mothNum):
    #load data from csv file
    loadPath =  f'~/Documents/GitHub/AL_generative/ALdata/{mothNum}_spontaneous.csv'

    df = pd.read_csv(loadPath, header = 0)
    
    
    #take samples with stimuli giving largest pre_stim value in first row
    # stimuli = argMinPreStim(mothNum)
    # Df = df.loc[df['stimuli'] == stimuli]
    

    tempDf = []
    tempName = []
    tempNeuron = []
    tempStimuli = []

    neuronCols = list(df.columns[3:])
    stimuli = df['stimuli'][0]
    
    for neuron in neuronCols:
        curArr,nameArr = readData(df[neuron],mothNum)
        tempDf += curArr
        tempName += nameArr
        tempNeuron += [neuron] * len(curArr)
        tempStimuli += [stimuli] * len(curArr)

    #add an entire column of mothNums at the end of tempDf
    #tempDf = np.array(tempDf)
    #mothNums = np.full((tempDf.shape[0],1),mothNum)
    #tempDf = np.concatenate((tempDf,mothNums),axis = 1)
    # print(tempName)

    return tempDf,tempName,tempNeuron,tempStimuli


#start collecting data for logistic regression
def collectModelData(mothNames):
    #for each moth, for each trial, for each neuron, render 9 parameters
    totalDf = []
    totalName = []
    totalNeuron = []
    totalStimuli = []
    for mothName in mothNames:
        mothDf,mothNameArr,mothNeuronArr,mothStimuliArr = loadData(mothName)
        print(f"########current number of rows: {len(mothDf)}########")
        totalDf += mothDf
        totalName += mothNameArr
        totalNeuron += mothNeuronArr
        totalStimuli += mothStimuliArr
    
    print("############all data loaded############")
    #print row number of totalDf
    print(f"############totalDf row number{len(totalDf)}############")

    return totalDf,totalName,totalNeuron,totalStimuli

#GIT data for unsupervised learning
mothNames = ['070906', '070913', '070921', '070922', '070924_1', '070924_2', '071002']
totalDf,totalName,totalNeuron,totalStimuli = collectModelData(mothNames)


########current number of rows: 12########
########current number of rows: 7########
########current number of rows: 14########
########current number of rows: 15########
########current number of rows: 16########
########current number of rows: 15########
########current number of rows: 14########
############all data loaded############
############totalDf row number93############


In [3]:
display(totalDf)

[array([17.1437, 17.1638, 17.1702, 17.2046, 17.2188, 17.2457, 17.2694,
        17.4844, 17.4965, 17.5062, 17.5543, 17.6392, 17.6652, 17.7928,
        17.8174, 18.8824, 18.9075, 19.2215, 19.2443, 19.2923, 19.696 ,
        19.7037, 19.805 ]),
 array([16.0076, 16.0728, 16.0798, 16.1174, 16.1687, 16.1875, 16.2199,
        16.2785, 16.4193, 16.4347, 16.5062, 16.5185, 16.5305, 16.5346,
        16.5529, 16.7391, 17.0222, 17.0859, 17.1523, 17.2681, 17.2783,
        17.3007, 17.304 , 17.3215, 17.3702, 17.4234, 17.4406, 17.5147,
        17.5316, 17.5449, 17.6216, 17.6361, 17.6947, 17.752 , 17.7748,
        17.7995, 17.8075, 17.9003, 18.701 , 18.7601, 18.8643, 18.9433,
        19.3699, 19.6723]),
 array([19.4153, 19.6343]),
 array([19.4293]),
 array([16.0835, 16.1781, 16.1951, 16.4575, 16.6217, 16.6641, 17.0181,
        17.0251, 17.1247, 17.1419, 17.1899, 17.2221, 17.2852, 17.5057,
        17.6412, 17.6589, 17.6781, 17.8487, 17.8693, 17.9497, 18.1114,
        18.2192, 18.458 , 18.5024, 18.5323, 1

Spike Plotting (Bea, Ctl, totalDf)

In [4]:
# #load file from _prep.mat
# def load_range_data(mothName,neuronName,stimuli):
#     #confirm time range from timestamps csv 
#     timestamps_csv_filename = f'~/Documents/GitHub/AL_generative/ALdata/timestamps_{mothName}.csv'
#     timestamp_start = pd.read_csv(timestamps_csv_filename,header=0).iloc[0][stimuli]
#     #2 seconds after stimulus
#     timestamp_end = timestamp_start + 2

#     #load data from _prep.mat file
#     prep_filename = f'../ALdata/{mothName}_prep.mat'
#     prep = loadmat(prep_filename)
#     prep_timestamps = list(i[0] for i in prep[neuronName])
#     #crop the part of prep_timestamps where the time is larger than timestamp_start and smaller than timestamp_end
#     cropped_prep_timestamps = [i for i in prep_timestamps if i >= timestamp_start and i <= timestamp_end]

#     return timestamp_start,timestamp_end,cropped_prep_timestamps

# def y_spikes(x_start,x_end,timestamps):
#     x_bot = np.linspace(x_start,x_end,200000)

#     y = []
#     for x in x_bot:
#         x = round(x,4)
#         if x in timestamps:
#             y.append(1)
#         else:
#             y.append(0)

#     return x_bot,y

# def plot_spike(index):
#     fig,axs = plt.subplots(3,1,figsize = (10,15))
    
#     #totalDf
#     x_bot,y = y_spikes(0,20,totalDf[index])

#     #set x y label
#     axs[0].plot(x_bot,y)
#     axs[0].set_xlabel('time')
#     axs[0].set_ylabel('spike')
#     axs[0].set_title(f"spike train of unlabeled test data {totalName[index]}_{totalNeuron[index]}_Spontaneous")

#     #Bea
#     timestamp_start,timestamp_end,timestamps = load_range_data(totalName[index],totalNeuron[index],'Bea')
#     # display(timestamp_start,timestamp_end,timestamps)
#     x_bot,y = y_spikes(timestamp_start,timestamp_end,timestamps)
#     axs[1].plot(x_bot,y)
#     axs[1].set_xlabel('time')
#     axs[1].set_ylabel('spike')
#     axs[1].set_title(f"spike train of Bea {totalName[index]}_{totalNeuron[index]}_Bea")

#     #Ctl
#     timestamp_start,timestamp_end,timestamps = load_range_data(totalName[index],totalNeuron[index],'Ctl')
#     x_bot,y = y_spikes(timestamp_start,timestamp_end,timestamps)
#     # display(timestamp_start,timestamp_end,timestamps)
#     axs[2].plot(x_bot,y)
#     axs[2].set_xlabel('time')
#     axs[2].set_ylabel('spike')
#     axs[2].set_title(f"spike train of Ctl {totalName[index]}_{totalNeuron[index]}_Ctl")

#     # plt.show()
#     fig.savefig(f"bea_ctl_unlabeled_data_spikes_plots/bea_ctl_unlabeled_data_spikes_{totalName[index]}_{totalNeuron[index]}.pdf")

# for i in range(len(totalDf)):
#     plot_spike(i)